In [1]:
import utils
import models.categorical
import math
import pickle
import numpy as np
import pandas as pd
from datetime import date

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
file_dir = "logreg/pct/"
y_var = "LONG_SHORT"

for future in tqdm(utils.futuresList):
    print(future)
    
    # generate X vars
    X_vars = utils.generate_X_vars(future, linearise=False, tech_indicators=[], macro_indicators=False)
    
    models.categorical.save_meta_predictions(
        path = file_dir, metric="opp_cost_SMA", model_fn = LogisticRegression, future = future,
        X_vars = X_vars, y_var = y_var, model_name = "logreg"
    )

  0%|          | 0/88 [00:00<?, ?it/s]

F_AD


  0%|          | 0/88 [00:00<?, ?it/s]


In [4]:
preds

In [ ]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(logregParams))
y_var = "LONG_SHORT"
file_dir = "logreg/pct/"

for future in tqdm(utils.futuresList):
    print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=[], \
                                   macro_indicators=False)
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]
    
    # prepare collated results
    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        model = LogisticRegression(**param_set)
        win_results, agg_results = models.categorical.walk_forward(
            model = model, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)